In [6]:
import os

WIKIPEDIA_ROOT = "/media/jeff/Data/data/wikipedia"
WIKIPEDIA_DL = os.path.join(WIKIPEDIA_ROOT, 'dl')

In [11]:
import bs4
import os
from urllib.parse import urlparse
import urllib
import posixpath

WIKIPEDIA_URL = 'https://dumps.wikimedia.org/'
WIKIPEDIA_LANG = 'enwiki'

from bs4 import BeautifulSoup
base_url = posixpath.join(WIKIPEDIA_URL, WIKIPEDIA_LANG)
index = urllib.request.urlopen(base_url).read()
soup_index = BeautifulSoup(index, 'html.parser')
# Find the links on the page
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]
dumps

['../',
 '20201201/',
 '20201220/',
 '20210101/',
 '20210120/',
 '20210201/',
 '20210220/',
 '20210301/',
 'latest/']

# Choose the Dump to Download

In [14]:
dump_url = posixpath.join(base_url, '20210301')

print(dump_url)
# Retrieve the html
dump_html = urllib.request.urlopen(dump_url).read()
# Convert to a soup
soup_dump = BeautifulSoup(dump_html, 'html.parser')
# Find list elements with the class file
files = soup_dump.find_all('li', {'class': 'file'})

https://dumps.wikimedia.org/enwiki/20210301


In [15]:
files

[<li class="file"><a href="/enwiki/20210301/enwiki-20210301-pages-articles-multistream.xml.bz2">enwiki-20210301-pages-articles-multistream.xml.bz2</a> 18.0 GB</li>,
 <li class="file"><a href="/enwiki/20210301/enwiki-20210301-pages-articles-multistream-index.txt.bz2">enwiki-20210301-pages-articles-multistream-index.txt.bz2</a> 219.9 MB</li>,
 <li class="file"><a href="/enwiki/20210301/enwiki-20210301-pages-articles-multistream1.xml-p1p41242.bz2">enwiki-20210301-pages-articles-multistream1.xml-p1p41242.bz2</a> 235.7 MB</li>,
 <li class="file"><a href="/enwiki/20210301/enwiki-20210301-pages-articles-multistream-index1.txt-p1p41242.bz2">enwiki-20210301-pages-articles-multistream-index1.txt-p1p41242.bz2</a> 221 KB</li>,
 <li class="file"><a href="/enwiki/20210301/enwiki-20210301-pages-articles-multistream2.xml-p41243p151573.bz2">enwiki-20210301-pages-articles-multistream2.xml-p41243p151573.bz2</a> 318.1 MB</li>,
 <li class="file"><a href="/enwiki/20210301/enwiki-20210301-pages-articles-mult

In [4]:
for items in soup_dump.find_all('li', {'class': 'file'}):
    for child in items.children:
        if isinstance(child,bs4.element.Tag):
            f = child.get('href', default=None)
            if f and '-stub' not in f and '-current' in f and 'xml-' in f:
                url = urllib.parse.urljoin(WIKIPEDIA_URL, f)
                a = urlparse(url)
                target_file = os.path.join(WIKIPEDIA_DL,os.path.basename(a.path))
                print(url)
                urllib.request.urlretrieve(url, target_file)


https://dumps.wikimedia.org/enwiki/20200620/enwiki-20200620-pages-meta-current1.xml-p1p30303.bz2
https://dumps.wikimedia.org/enwiki/20200620/enwiki-20200620-pages-meta-current2.xml-p30304p88444.bz2
https://dumps.wikimedia.org/enwiki/20200620/enwiki-20200620-pages-meta-current3.xml-p88445p200509.bz2
https://dumps.wikimedia.org/enwiki/20200620/enwiki-20200620-pages-meta-current4.xml-p200510p352689.bz2
https://dumps.wikimedia.org/enwiki/20200620/enwiki-20200620-pages-meta-current5.xml-p352690p565313.bz2
https://dumps.wikimedia.org/enwiki/20200620/enwiki-20200620-pages-meta-current6.xml-p565314p892912.bz2
https://dumps.wikimedia.org/enwiki/20200620/enwiki-20200620-pages-meta-current7.xml-p892913p1268691.bz2


KeyboardInterrupt: 